In [2]:
import scanpy as sc
import numpy as np
import os
import pandas as pd
from pathlib import Path
from scipy import sparse

In [12]:
rna_obs = pd.read_csv("/home/bram/jointomicscomp/data/CELL/pbmc_multimodal_RNA_5000MAD_obs.csv", index_col=0)
protein_obs = pd.read_csv("/home/bram/jointomicscomp/data/CELL/pbmc_multimodal_ADT_5000MAD_obs.csv", index_col=0)


In [20]:
rna_samples = rna_obs.index.values.astype(str)
protein_samples = protein_obs.index.values.astype(str)
assert np.array_equal(rna_samples, protein_samples)

In [40]:
protein_obs

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase
L1_AAACCCAAGAAACTCA,7535.0,217,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1
L1_AAACCCAAGACATACA,6013.0,209,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1
L1_AAACCCACAACTGGTT,6620.0,213,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S
L1_AAACCCACACGTACTA,3567.0,202,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1
L1_AAACCCACAGCATACT,6402.0,215,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1
...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4170.0,197,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S
E2L8_TTTGTTGGTGTGCCTG,6927.0,209,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1
E2L8_TTTGTTGGTTAGTTCG,4222.0,206,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S
E2L8_TTTGTTGGTTGGCTAT,6063.0,217,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1


In [68]:
# Now fetch and store columns we're interested in (celltypes, samples)
for ct in ['celltype.l1', 'celltype.l2', 'celltype.l3']:
    cellTypes = np.unique(rna_obs[ct].values.astype(str))
    cellType = np.zeros(len(rna_samples)).astype(int)

    for i in range(len(rna_obs[ct].values)):
        cellType[i] = np.where(cellTypes == rna_obs[ct].values[i])[0][0]

    print(cellType.dtype)

int64
int64
int64


In [56]:
celltype_l2

array(['4', '9', '11', ..., '1', '5', '4'], dtype='<U17')

In [2]:
rna_path = "/home/bram/jointomicscomp/data/CELL/pbmc_multimodal_RNA.h5ad"
adt_path = "/home/bram/jointomicscomp/data/CELL/pbmc_multimodal_ADT.h5ad"

In [3]:
# First process RNA, then protein
adata = sc.read_h5ad(rna_path)

/home/bram/miniconda3/envs/vae/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [4]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)

In [5]:
adata.obs

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase
L1_AAACCCAAGAAACTCA,7535.0,217,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1
L1_AAACCCAAGACATACA,6013.0,209,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1
L1_AAACCCACAACTGGTT,6620.0,213,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S
L1_AAACCCACACGTACTA,3567.0,202,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1
L1_AAACCCACAGCATACT,6402.0,215,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1
...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4170.0,197,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S
E2L8_TTTGTTGGTGTGCCTG,6927.0,209,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1
E2L8_TTTGTTGGTTAGTTCG,4222.0,206,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S
E2L8_TTTGTTGGTTGGCTAT,6063.0,217,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1


In [6]:
data = adata.X[:, adata.var.highly_variable]

In [7]:
np.save("/home/bram/Desktop/testdata.npy", data.astype(np.float64))

In [8]:
data.indices.shape

(92628629,)

In [12]:
npdata = np.load("/home/bram/Desktop/testdata.npy", allow_pickle=True)

In [18]:
okok.item()

ValueError: can only convert an array of size 1 to a Python scalar